In [1]:
import pandas as pd
import openpyxl
import xlrd
import numpy as np
from numpy import nan
import glob
import datetime
from math import cos, sin, pi 

In [2]:

## we want no cientific notation numbers:
np.set_printoptions(suppress=True)

In [3]:

def df_monthly_total_production(list_directories):
    
    ##Here we define all our empty arrays determined to be our future features:
    seasonability_1 = np.empty(0)
    seasonability_2 = np.empty(0)
    
    pmf = np.empty(0)
    empty_array = np.empty(0)
    
    trend = np.empty(0)

    number_months = range(1,13)
    
    for year in list_directories:
        for data in year:
                ## we open each year and then each excel related to each month:
                df = pd.read_excel(data, engine='openpyxl')
                fm = empty_array
                
                ## we filter only the bill lines which are commercial invices (values 'F' of the 'TipoDoc' Column):
                df_f = df.loc[df['TipoDoc'].isin(['F'])]
                
                ## we filter the date info inside the fecha column:
                df['year'] = pd.DatetimeIndex(df['Fecha']).year
                df['month'] = pd.DatetimeIndex(df['Fecha']).month
                
                ## in order to know when a line is part of the same bill we stablished that the value for a row
                ## in the column 'Linea Doc' it has to be different in the next row when they share the same value in 
                ## the column 'Documento' (ID number of the document), that's why we use the pandas command shift:
                df['siguiente_linea'] = df['LineaDoc'].shift(-1)
                df['siguiente_doc'] = df['Documento'].shift(-1)

                ## now in the last row we will have a NaN value, we should change that:
                df.loc[np.isnan(df["siguiente_linea"]), 'siguiente_linea'] = df['LineaDoc'][-1:]
                df.loc[np.isnan(df["siguiente_doc"]), 'siguiente_doc'] = df['Documento'][-1:]
                
                df['siguiente_linea'] = df['siguiente_linea'].astype('int64')
                df['siguiente_doc'] = df['siguiente_doc'].astype('int64')
                
                ## let's try to use the pandas' command .drop() in order to implement the mentioned restriction:
                df = df.drop(df[(df['Documento'] == df['siguiente_doc']) & (df['LineaDoc'] == df['siguiente_linea'])].index)
                
                ## we sum up the value of the left rows (commercial invoices), which is the value related to the column 'Importe':
                fm = df['Importe'].sum()
                
                ## we append to the general array the result of the fourth weeks of the loaded month:
                pmf = np.append(pmf, fm)

        ## we store the trend info (passage of time):
        ## we store the year:
        for month in number_months:
            y = df['year'][-1:]
            trd = y + (month - 1)/12

            ## then we store this trend number into an array:
            trend = np.append(trend, trd)
            
            ## we save the information of the month:
            dt_circ_base = (month/12)*2*pi # entre 0 y 2*pi
            features_dt_circ = [cos(dt_circ_base), sin(dt_circ_base)] # 2 valores, a rellenar en 2 columnas de features
            
            ## we store the info of the cos:
            seasonability_1 = np.append(seasonability_1, features_dt_circ[0])

            ## and the info  of the sin:
            seasonability_2 = np.append(seasonability_2, features_dt_circ[1])
                      
    ##here after the iterations are done we store the arrays generated:
    yTrain = pd.DataFrame({'target':pmf})
    
    xTrain = pd.DataFrame({'seasonability_circ_cos':seasonability_1,'seasonability_circ_sin':seasonability_2, 'time':trend, 'y_t-1':yTrain.target.shift(1)})
    return yTrain, xTrain


In [23]:
tablas20_1 = glob.glob("C:/Users/Javier/Documents/GIT/Repositories/TFM-Textil-Santanderina/data/raw/2015*.xlsx")
tablas20_2 = glob.glob("C:/Users/Javier/Documents/GIT/Repositories/TFM-Textil-Santanderina/data/raw/2016*.xlsx")
tablas20_3 = glob.glob("C:/Users/Javier/Documents/GIT/Repositories/TFM-Textil-Santanderina/data/raw/2017*.xlsx")
tablas20_4 = glob.glob("C:/Users/Javier/Documents/GIT/Repositories/TFM-Textil-Santanderina/data/raw/2018*.xlsx")
tablas20_5 = glob.glob("C:/Users/Javier/Documents/GIT/Repositories/TFM-Textil-Santanderina/data/raw/2019*.xlsx")
tablas20_all = list([tablas20_1,tablas20_2,tablas20_3, tablas20_4, tablas20_5])

In [24]:
y, x = df_monthly_total_production(tablas20_all)

In [39]:
y[-24:-12].sum()

target    85295382.91
dtype: float64